# All In One

This notebook showcases some of the most common use cases to interact with a `GeoServer` server using its REST API through the `geoserver-py` Python package.

## Imports

Let's start by importing the required libraries:

In [1]:
from pathlib import Path
from geoserver import GeoServer

## Config

We'll set up the configuration for the notebook:

In [2]:
# Directory containing sample data
EXAMPLES_DIR = Path("../examples")
assert EXAMPLES_DIR.exists(), f"The directory {EXAMPLES_DIR} does not exist."

## Connection

Connect to the GeoServer instance using the `GeoServer` object.
You can pass additional parameters to the constructor, such as the `username`, `password` and other [`requests`]() supported parameters.

In [3]:
geoserver = GeoServer(
    "http://localhost:8080/geoserver", 
    username="admin", 
    password="geoserver"
)

### Setup

If the demo workspace already exists, we will delete it so that the demo can be run multiple times.

In [4]:
if geoserver.workspace_exists(workspace="demo"):
    geoserver.delete_workspace(workspace="demo", recurse=True)

## Workspaces

### List existing workspaces

To list the existing workspaces, we can use the `get_workspaces` method.

In [5]:
geoserver.get_workspaces()

{'workspaces': ''}

It is also possible to return the result as XML format using:

In [6]:
workspace = geoserver.get_workspaces(format="xml")  # Default is "json"
print(workspace)

<workspaces/>


### Create a new workspace

To create a new workspace, use the `create_workspace` method. This method will return a success message if the workspace is created successfully.

In [7]:
# Using JSON format
body = {"workspace": {"name": "demo"}}
# Using XML format
body = "<workspace><name>demo</name></workspace>"


geoserver.create_workspace(body=body)

'Created'

### Select a workspace

To select a specific workspace, use the `get_workspace` method.


In [8]:
geoserver.get_workspace(workspace="demo")

{'workspace': {'name': 'demo',
  'isolated': False,
  'dateCreated': '2024-06-05 22:58:40.292 UTC',
  'dataStores': 'http://localhost:8080/geoserver/rest/workspaces/demo/datastores.json',
  'coverageStores': 'http://localhost:8080/geoserver/rest/workspaces/demo/coveragestores.json',
  'wmsStores': 'http://localhost:8080/geoserver/rest/workspaces/demo/wmsstores.json',
  'wmtsStores': 'http://localhost:8080/geoserver/rest/workspaces/demo/wmtsstores.json'}}

### Update a workspace

To update a workspace, use the `update_workspace` method.

In [9]:
# Using JSON format
body = {"workspace": {"name": "demo", "enabled": "true"}}
# Using XML format
body = "<workspace><name>demo</name><enabled>true</enabled></workspace>"


geoserver.update_workspace(workspace="demo", body=body)

'Updated'

### Remove a workspace

To remove a workspace, use the `delete_workspace` method. This method will return a success message if the workspace is removed successfully. If the workspace is not empty, an error will be raised. To remove a workspace and all its contents, use the `recurse=True` argument.

```python
geoserver.delete_workspace(workspace="demo", recurse=True)
```

## Datastores

This section will guide you on how to interact with data stores in the GeoServer instance.

### List existing data stores

To view the data stores in the workspace, use the `get_data_stores` method:

In [10]:
geoserver.get_data_stores(workspace="demo")

{'dataStores': ''}

### Create a new data stores

There are different ways to create / add data in a GeoServer instance. You can either upload diretly your data (from a local file, external URL, PostGIS, etc.) or create a new data store from a data already present in the GeoServer instance (e.g. you can reference a file that is already present in the `data` directory of the geoserver).

#### From a local shapefile

To upload a data store from a local shapefile, use the `upload_data_store` method. This method will return a success message if the data store is created successfully.

In [11]:
# Get the shapefile
file_path = Path("../tests/data/states.shp")
assert file_path.exists(), f"File not found: {file_path.as_posix()!r}"


# If the store already exists, it will be overwritten
geoserver.upload_data_store(file=file_path, workspace="demo", store="states")
# If the `store` parameter is not provided, the name of the store will be the same as the file name
geoserver.upload_data_store(file=file_path, workspace="demo")

'Created'

#### From a file already present in the GeoServer instance

Now that we have uploaded a file `states.shp` to the GeoServer instance, we can create a new data store from this file.

In [12]:
# Using JSON format
body={
    "dataStore": {
        "name": "new_states",
        "connectionParameters": {
            "entry": [
                {"@key":"url", "$": "file:data/demo/states/states.shp"},
                {"@key":"filetype", "$": "shapefile"},
            ]
        }
    }
}


# Using XML format
body = """
<dataStore>
    <name>new_states</name>
    <connectionParameters>
        <entry key="url">file:data/demo/states/states.shp</entry>
        <entry key="filetype">shapefile</entry>
    </connectionParameters>
</dataStore>
"""


geoserver.create_data_store(workspace="demo", body=body)

'Created'

#### From a PostGIS database

You can also add a PostGIS data store to the GeoServer instance. To do this, you need to provide the database connection details.

In [13]:
# Using JSON format
body = {
    "dataStore": {
        "name": "postgis",
        "description": "PostGIS connection",
        "connectionParameters": {
            "host": "postgis",
            "port": "5432",
            "database": "db",
            "user": "admin",
            "passwd": "postgres",
            "dbtype": "postgis",
            "schema": "public",
            "Expose primary keys": "true",
            "Loose bbox": "true",
            "Estimated extends": "true",
            "fetch size": "1000",
            "Max open prepared statements": "50",
            "preparedStatements": "false",
            "validate connections": "true",
            "validate connections on borrow": "true",
            "validate connections on return": "true",
            "Connection timeout": "20",
            "Eviction run periodicity": "3600",
            "Min evictable idle time": "300",
            "Max active": "50",
            "Max idle": "10",
            "Max wait": "10000",
            "Test on borrow": "true",
            "Test while idle": "true",
            "Time between eviction runs": "60000",
        }
    }
}

# Using XML format
body = """
<dataStore>
    <name>postgis</name>
    <description>PostGIS connection</description>
    <connectionParameters>
        <entry key="host">postgis</entry>
        <entry key="port">5432</entry>
        <entry key="database">db</entry>
        <entry key="user">admin</entry>
        <entry key="passwd">postgres</entry>
        <entry key="dbtype">postgis</entry>
        <entry key="schema">public</entry>
        <entry key="Expose primary keys">true</entry>
        <entry key="Loose bbox">true</entry>
        <entry key="Estimated extends">true</entry>
        <entry key="fetch size">1000</entry>
        <entry key="Max open prepared statements">50</entry>
        <entry key="preparedStatements">false</entry>
        <entry key="validate connections">true</entry>
        <entry key="validate connections on borrow">true</entry>
        <entry key="validate connections on return">true</entry>
        <entry key="Connection timeout">20</entry>
        <entry key="Eviction run periodicity">3600</entry>
        <entry key="Min evictable idle time">300</entry>
        <entry key="Max active">50</entry>
        <entry key="Max idle">10</entry>
        <entry key="Max wait">10000</entry>
        <entry key="Test on borrow">true</entry>
        <entry key="Test while idle">true</entry>
        <entry key="Time between eviction runs">60000</entry>
    </connectionParameters>
</dataStore>
"""


geoserver.create_data_store(workspace="demo", body=body)

'Created'

Once added, you should also publish the feature types contained in the data store. Use the `create_feature_type` method to do this.

In [14]:
# Using JSON format
body = {
    "featureType": {
        "name": "jamoat_db",
        "title": "jamoat_db",
        "advertised": "true",
    }
}
# Using XML format
body = """
<featureType>
    <name>jamoat_db</name>
    <title>jamoat_db</title>
    <advertised>true</advertised>
</featureType>
"""


geoserver.create_feature_type(workspace="demo", store="postgis", body=body)

'Created'

### Select a data store

Now that we created a datastore from a vector source, we can list it using the `get_data_stores` method.

In [15]:
geoserver.get_data_store(workspace="demo", store="states")

{'dataStore': {'name': 'states',
  'type': 'Shapefile',
  'enabled': True,
  'workspace': {'name': 'demo',
   'href': 'http://localhost:8080/geoserver/rest/workspaces/demo.json'},
  'connectionParameters': {'entry': [{'@key': 'namespace', '$': 'http://demo'},
    {'@key': 'url',
     '$': 'file:/opt/geoserver/data_dir/data/demo/states/states.shp'}]},
  '_default': False,
  'dateCreated': '2024-06-05 22:58:40.399 UTC',
  'dateModified': '2024-06-05 22:58:40.412 UTC',
  'disableOnConnFailure': False,
  'featureTypes': 'http://localhost:8080/geoserver/rest/workspaces/demo/datastores/states/featuretypes.json'}}

### Clear data caches

You can clear the data caches using the `clear_data_store_cache` method.

In [16]:
geoserver.reset_data_store_caches(workspace="demo", store="states")

'OK'

### Update a data store

To update a data store, use the `update_data_store` method.

In [17]:
# Using JSON format
body = {
    "dataStore": {
        "name": "states",
    }
}

# Using XML format
body = "<dataStore><name>states</name></dataStore>"


geoserver.update_data_store(workspace="demo", store="states", body=body)

'Updated'

### Remove a data store

To delete a datastore, use the following command:

```python
geoserver.delete_data_store(workspace="demo", datastore="states")
```

## Coverage Stores

Coverage stores are used to store raster data. A coverage is a raster based data which originates from a coverage store.

### List existing coverage stores

To list the existing coverage stores, use the `get_coverage_stores` method.

In [18]:
geoserver.get_coverage_stores(workspace="demo")

{'coverageStores': ''}

### Create a new coverage store

You can create a coverage store from a local raster file. Use the `upload_coverage_store` method to create a new coverage store.

In [19]:
file_path = EXAMPLES_DIR / "rasters" / "raster1.tif"
assert file_path.exists(), f"File not found: {file_path.as_posix()!r}"


# If the store already exists, it will be overwritten
geoserver.upload_coverage_store(file=file_path, format="geotiff", workspace="demo", store="raster1")
# If the `store` parameter is not provided, the name of the store will be the same as the file name
geoserver.upload_coverage_store(file=file_path, format="geotiff", workspace="demo")

'Created'

### Select a coverage store

To select a specific coverage store, use the `get_coverage_store` method.

In [20]:
geoserver.get_coverage_store(workspace="demo", store="raster1")

{'coverageStore': {'name': 'raster1',
  'type': 'GeoTIFF',
  'enabled': True,
  'workspace': {'name': 'demo',
   'href': 'http://localhost:8080/geoserver/rest/workspaces/demo.json'},
  '_default': False,
  'dateCreated': '2024-06-05 22:58:40.963 UTC',
  'dateModified': '2024-06-05 22:58:41.12 UTC',
  'disableOnConnFailure': False,
  'url': 'file:data/demo/raster1/raster1.tif',
  'coverages': 'http://localhost:8080/geoserver/rest/workspaces/demo/coveragestores/raster1/coverages.json'}}

### Update a coverage store

In [21]:
# Using JSON format
body = {
    "coverageStore": {
        "name": "raster1",
    }
}

# Using XML format
body = """
<coverageStore>
    <name>raster1</name>
</coverageStore>
"""


geoserver.update_coverage_store(workspace="demo", store="raster1", body=body)

'Updated'

### Remove a coverage store

To remove a coverage store, use the `delete_coverage_store` method.

```python
geoserver.delete_coverage_store(workspace="demo", store="raster1", recurse=True)
```

## Coverages

Coverages are automatically created when a coverage store is created.

### List existing coverages

To list the existing coverages, use the `get_coverages` method.

In [22]:
geoserver.get_coverages(workspace="demo")

{'coverages': {'coverage': [{'name': 'raster1',
    'href': 'http://localhost:8080/geoserver/rest/workspaces/demo/coverages/raster1.json'}]}}

### Select a coverage

To select a specific coverage, use the `get_coverage` method.

In [23]:
geoserver.get_coverage(workspace="demo", coverage="raster1")

{'coverage': {'name': 'raster1',
  'nativeName': 'raster1',
  'namespace': {'name': 'demo',
   'href': 'http://localhost:8080/geoserver/rest/namespaces/demo.json'},
  'title': 'raster1',
  'description': 'Generated from GeoTIFF',
  'keywords': {'string': ['raster1', 'WCS', 'GeoTIFF']},
  'nativeCRS': {'@class': 'projected',
   '$': 'PROJCS["WGS 84 / UTM zone 42N", \n  GEOGCS["WGS 84", \n    DATUM["World Geodetic System 1984", \n      SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], \n      AUTHORITY["EPSG","6326"]], \n    PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Geodetic longitude", EAST], \n    AXIS["Geodetic latitude", NORTH], \n    AUTHORITY["EPSG","4326"]], \n  PROJECTION["Transverse_Mercator", AUTHORITY["EPSG","9807"]], \n  PARAMETER["central_meridian", 69.0], \n  PARAMETER["latitude_of_origin", 0.0], \n  PARAMETER["scale_factor", 0.9996], \n  PARAMETER["false_easting", 500000.0], \n  PARAMET

## Styles

The REST API allows you to list, create, upload, update, and delete styles in GeoServer.

### Listing all styles

To list all the styles in the GeoServer instance, use the `get_styles` method.

In [24]:
geoserver.get_styles()

{'styles': {'style': [{'name': 'generic',
    'href': 'http://localhost:8080/geoserver/rest/styles/generic.json'},
   {'name': 'line',
    'href': 'http://localhost:8080/geoserver/rest/styles/line.json'},
   {'name': 'point',
    'href': 'http://localhost:8080/geoserver/rest/styles/point.json'},
   {'name': 'polygon',
    'href': 'http://localhost:8080/geoserver/rest/styles/polygon.json'},
   {'name': 'raster',
    'href': 'http://localhost:8080/geoserver/rest/styles/raster.json'}]}}

### Retrieve a style

To get a specific style, use the `get_style` method.

In [25]:
geoserver.get_style(style="point")

{'style': {'name': 'point',
  'format': 'sld',
  'languageVersion': {'version': '1.0.0'},
  'filename': 'default_point.sld'}}

### Creating a style

You can create a new style on the server in two ways. In the first way, the creation is done in two steps: the style entry is created in the catalog, and then the style content is uploaded.
The second way can add the style to the server in a single step by uploading a file containing the style content.


#### Create a new style in two steps

In [26]:
# Using XML format
body = """
<style>
    <name>raster1</name>
    <filename>raster1.sld</filename>
</style>
"""

geoserver.create_style(body=body, workspace="demo")

'Created'

In [27]:
# Get the styles associated to the raster layer
file_path = EXAMPLES_DIR / "styles" / "raster1.sld"
assert file_path.exists(), f"File not found: {file_path.as_posix()!r}"


with open(file_path, "r") as file:
    body = file.read()

    
# geoserver.update_style(style="raster1", body=body, workspace="demo")

#### Uploading a style from an SLD file

In [28]:
# Get the styles associated to the raster layer
file_path = EXAMPLES_DIR / "styles" / "raster1.sld"
assert file_path.exists(), f"File not found: {file_path.as_posix()!r}"


# geoserver.upload_style(file=file_path, workspace="demo", style="raster1_v2")

### Changing an existing style

To update a style, use the `update_style` method.

In [29]:
# Get the styles associated to the raster layer
file_path = EXAMPLES_DIR / "styles" / "raster1.sld"
assert file_path.exists(), f"File not found: {file_path.as_posix()!r}"


with open(file_path, "r") as file:
    body = file.read()

    
geoserver.update_style(style="raster1", body=body, workspace="demo")

'Updated'

### Downloading a style

The SLD itself can be downloaded using the `download_style` method.

In [30]:
xml = geoserver.download_style(style="raster1", workspace="demo")
print(xml)

<?xml version="1.0" encoding="UTF-8"?><sld:StyledLayerDescriptor xmlns="http://www.opengis.net/sld" xmlns:sld="http://www.opengis.net/sld" xmlns:gml="http://www.opengis.net/gml" xmlns:ogc="http://www.opengis.net/ogc" version="1.0.0">
  <sld:NamedLayer>
    <sld:Name>raster1</sld:Name>
    <sld:UserStyle>
      <sld:Name>raster1</sld:Name>
      <sld:FeatureTypeStyle>
        <sld:Name>name</sld:Name>
        <sld:Rule>
          <sld:RasterSymbolizer>
            <sld:ChannelSelection>
              <sld:GrayChannel>
                <sld:SourceChannelName>1</sld:SourceChannelName>
                <sld:ContrastEnhancement>
                  <sld:GammaValue>1.0</sld:GammaValue>
                </sld:ContrastEnhancement>
              </sld:GrayChannel>
            </sld:ChannelSelection>
            <sld:ColorMap type="values">
              <sld:ColorMapEntry color="#13de13" quantity="1" label="class 1"/>
              <sld:ColorMapEntry color="#4d4def" quantity="2" label="class 2"/>
  

### Deleting a style

To delete a style, use the `delete_style` method. You can use the `purge=True` argument to remove the style and all its references.